In [1]:
import os 
import time 
import uuid 
import cv2
import tensorflow as tf 
import json 
import numpy as np
from matplotlib import pyplot as plt

# 1. Collecting Images

In [2]:
IMAGES_PATH = os.path.join('data', 'images')
# total images: 120
NUM_IMGS = 20

In [9]:
capture = cv2.VideoCapture(0)
for img_idx in range(NUM_IMGS):
    ret, frame = capture.read() 
    img_name = os.path.join(IMAGES_PATH, f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(img_name, frame)
    cv2.imshow('frame', frame)
    time.sleep(0.5)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

Annotate images using LabelMe.

In [ ]:
!labelme 

# 2. Limit GPU Memory Growth

In [19]:
# check for gpu(s)
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [20]:
tf.test.gpu_device_name()

'/device:GPU:0'

# 3. Build and Review Dataset